In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
!pip install snac

In [2]:
from unsloth import FastLanguageModel
from snac import SNAC
from IPython.display import display, Audio
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
START_TOKEN = torch.tensor([[ 128259]], dtype=torch.int64) # Start of human
END_TOKENS = torch.tensor([[128009, 128260]], dtype=torch.int64) # End of text, End of human
TOKEN_TO_FIND = 128257
TOKEN_TO_REMOVE = 128258
CODER_VALUE = 128266

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "dkulemin/lora_orpheus_gothic",
    max_seq_length= 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.8.7: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.8.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
FastLanguageModel.for_inference(model)
None

In [6]:
snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")

In [7]:
snac_model.to("cpu")
None

In [8]:
def redistribute_codes(code_list):
  layer_1 = []
  layer_2 = []
  layer_3 = []
  for i in range((len(code_list)+1)//7):
    layer_1.append(code_list[7*i])
    layer_2.append(code_list[7*i+1]-4096)
    layer_3.append(code_list[7*i+2]-(2*4096))
    layer_3.append(code_list[7*i+3]-(3*4096))
    layer_2.append(code_list[7*i+4]-(4*4096))
    layer_3.append(code_list[7*i+5]-(5*4096))
    layer_3.append(code_list[7*i+6]-(6*4096))
  codes = [torch.tensor(layer_1).unsqueeze(0),
         torch.tensor(layer_2).unsqueeze(0),
         torch.tensor(layer_3).unsqueeze(0)]

  # codes = [c.to("cuda") for c in codes]
  audio_hat = snac_model.decode(codes)
  return audio_hat

In [9]:
def infer_one(model, tokenizer, snac_model, voice, prompt, temperature=0.6, top_p=0.95, repetition_penalty=1.1, num_return_sequences=1):
    input_ids = tokenizer("%s: %s" % (voice, prompt), return_tensors="pt").input_ids
    input_ids = torch.cat([START_TOKEN, input_ids, END_TOKENS], dim=1).to("cuda")
    attention_mask = torch.ones((1, input_ids.shape[1]), dtype=torch.int64).to("cuda")
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1200,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences,
        eos_token_id=TOKEN_TO_REMOVE,
        use_cache = True
    )

    token_indices = (generated_ids == TOKEN_TO_FIND).nonzero(as_tuple=True)
    if len(token_indices[1]) > 0:
        last_occurrence_idx = token_indices[1][-1].item()
        cropped_tensor = generated_ids[:, last_occurrence_idx+1:]
    else:
        cropped_tensor = generated_ids

    processed_rows = []

    for row in cropped_tensor:
        masked_row = row[row != TOKEN_TO_REMOVE]
        processed_rows.append(masked_row)

    code_lists = []

    for row in processed_rows:
        row_length = row.size(0)
        new_length = (row_length // 7) * 7
        trimmed_row = row[:new_length]
        trimmed_row = [t - CODER_VALUE for t in trimmed_row if t - CODER_VALUE > 0]
        code_lists.append(trimmed_row)

    my_samples = []
    for code_list in code_lists:
        samples = redistribute_codes(code_list)
        my_samples.append(samples)

    return my_samples

In [10]:
prompts = [
    "Привет, человек!",
    # Пачо
    # "Info_Pacho_GRD_FM_13_01"
    "Ну, сколько можно тебя ждать?",
    # "Info_Pacho_GRD_FM_13_03"
    "Ладно, все равно он сам сильно задерживается.",
    # "Info_Pacho_GRD_FM_END_13_01"
    "А куда это ты собрался?",
    # "Info_Pacho_GRD_FM_END_13_03"
    "Эй, парни, идите сюда! У нас предатель!",

    # "Info_Org_867_Raeuber_BDT_13_00"
    "Я получаю достаточно, как стражник лагеря.",
    # "Org_867_Raeuber_About_13_00"
    "Квентин убит.",
    # "Org_867_Raeuber_About_13_01"
    "Скорее всего его убили стражники из Старого лагеря.",
    # "Org_867_Raeuber_About_13_02"
    "Франко наш новый руководитель.",
    # "Org_867_Raeuber_About_13_03"
    "Все, как обычно.",
    # "Org_867_Raeuber_Banditencamp_13_01"
    "Это лагерь бандитов, парень.",
    # "Org_867_Raeuber_GUARD_Info_13_00"
    "Стой!",
    # "Org_867_Raeuber_GUARD_Info_13_02"
    "Тебе здесь нечего делать, парень.",
    # "Org_867_Raeuber_GoAway_Info_13_00"
    "Еще один придурок из Старого лагеря.",
    # "Org_867_Raeuber_Main_13_00"
    "Сейчас нас возглавляет Франко.",
    # "Org_867_Raeuber_Main_13_01"
    "Квентин главный в лагере.",
    # "Org_867_Raeuber_Organisators_13_01"
    "Больше нет.",
    # "Org_867_Raeuber_Organisators_13_02"
    "Магам воды и наемникам не нравилось, что мы совершаем набеги на конвои из Старого лагеря.",
    # "Org_867_Raeuber_Organisators_13_03"
    "Ларес предложил части воров уйти из Нового лагеря и посроить свой лагерь здесь.",
    # "Org_867_Raeuber_Organisators_13_04"
    "Недавно мы напали еще на один отряд стражников.",
    # "Org_867_Raeuber_Organisators_13_05"
    "В лагере. Пройди дальше через ущелье.",
    # "Org_867_Raeuber_Organisators_13_06"
    "Еще трое крутят болотник в пещере около Нового лагеря.",
    # "Org_867_Raeuber_WereIsMain_13_00"
    "Он в лагере. Просто иди дальше через ущелье.",
    # "Org_867_Raeuber_WereIsMain_13_01"
    "Он куда-то ушел из лагеря пару дней назад.",

    # Франко
    # "Info_Org_869_Raeuber_BDT_13_00"
    "А ты что думал? Он же глава банды.",
    # "Info_Org_869_Raeuber_TRADE_13_00"
    "Да, у меня есть куча товара с нашего склада.",

    # "SVM_13_YouAskedForIt"
    "Ты сам напросился!",
]

chosen_voice = "15" # None for single-speaker

In [ ]:
for prompt in prompts:
    try:
        samples = infer_one(model, tokenizer, snac_model, "13", prompt, num_return_sequences=3)
        print(prompt)
        for sample in samples:
            display(Audio(sample.detach().squeeze().to("cpu").numpy(), rate=24000))
    except:
        print("Failed for prompt:", prompt)

Привет, человек!


Ну, сколько можно тебя ждать?
